In [1]:
#Original Spacy Model
import spacy
#Spacy Model For identifying titles
import Model_50
#Importing Displacy
from spacy import displacy
#For Dates and Duration calculation
import datetime
import dateutil.parser
from datetime import date
import re
import datetime
import dateutil.parser as dparser
import datefinder

In [2]:
#For finding the path of our folder
import json
import os
import inspect
import sys
import pandas
import requests

In [3]:
#Libraries for extracting content from the pdfs
import pdfminer.settings
import six
import io
pdfminer.settings.STRICT = False
import pdfminer.layout
from pdfminer.image import ImageWriter
from pdfminer.layout import LAParams
from pdfminer.pdfinterp import PDFResourceManager
from pdfminer.converter import PDFPageAggregator
from pdfminer.pdfpage import PDFPage
from pdfminer.layout import LTTextBoxHorizontal
from pdfminer.layout import LTChar
from pdfminer.pdfinterp import PDFPageInterpreter

In [4]:
import time
nlp_title = spacy.load('Model_50')

In [5]:
def Spacy_Identified(text,nlp,line_number):
    return(list())

In [6]:
def find_if_work_experience_available(file_path,nlp,start_time):
    
    #Line number
    line_number = 0
    
    #Checking whether the work experience section is available
    counter_header = 0
    
    #If available, We have to assign the starting line number and ending line number to it for easy access
    line_number_start = 0
    line_number_end = 0
    
    #Opening the document
    document = open(file_path, 'rb')
    
    #Create resource manager
    rsrcmgr = PDFResourceManager()
    fake_file_handle = io.StringIO()
    
    # Set parameters for analysis.
    #line_overlap=0.5, char_margin=2.0, line_margin=0.5, word_margin=0.1, boxes_flow=0.5,detect_vertical=False, all_texts=False
    laparams = LAParams(line_overlap=0.5, char_margin=50.0, line_margin=0.5,word_margin=0.1, boxes_flow=0.5,
                         detect_vertical=False, all_texts=False)
    
    # Create a PDF page aggregator object.
    device = PDFPageAggregator(rsrcmgr, laparams=laparams)
    interpreter = PDFPageInterpreter(rsrcmgr, device)

    for page in PDFPage.get_pages(document,caching=True,check_extractable=True):
        interpreter.process_page(page)
        # receive the LTPage object for the page.
        layout = device.get_result()
        for element in layout:
            line_number+=1
            if isinstance(element, LTTextBoxHorizontal):
                text=element.get_text()
                text_nlp=nlp_title(text)  
                
                for t in text_nlp.ents:
                    if t.label_ == 'WORK_EXP_HEADING':
                        counter_header = 1
                        line_number_start = line_number
                        
                    elif t.label_ == 'HEADING':
                        if(counter_header == 1):
                            line_number_end = line_number
    
    print("--- %s seconds ---" % (time.time() - start_time))
    return(counter_header, line_number_start, line_number_end)

In [7]:
def dealing_with_work_experience_heading(line_number_start,line_number_end,file_path,start_time):
    
    #Line number initialization
    line_number = 0
    List_Org = []
    
    #print("Converting PDF to Text to extract features")
    document = open(file_path, 'rb')
    #Create resource manager
    rsrcmgr = PDFResourceManager()
    fake_file_handle = io.StringIO()
    # Set parameters for analysis.
    laparams = LAParams(line_overlap=0.5, char_margin=50.0, line_margin=0.5,word_margin=0.1, boxes_flow=0.5,
                         detect_vertical=False, all_texts=False)
    # Create a PDF page aggregator object.
    device = PDFPageAggregator(rsrcmgr, laparams=laparams)
    interpreter = PDFPageInterpreter(rsrcmgr, device)

    for page in PDFPage.get_pages(document,caching=True,check_extractable=True):
        interpreter.process_page(page)
        # receive the LTPage object for the page.
        layout = device.get_result()
        for element in layout:
            line_number+=1
            if isinstance(element, LTTextBoxHorizontal):
                text=element.get_text()
                if(line_number >= line_number_start and line_number <= line_number_end):
                    text=nlp_title(text)
                    lis=Spacy_Identified(text,nlp_title,line_number)
                    if(lis != None):
                        List_Org.append(lis)

    flatten=[l for subitem in List_Org for l in subitem]
    for l in flatten:
        year=calculate_duration(l[1])
        l[1].append(year)
        
    print("--- %s seconds ---" % (time.time() - start_time))
    return(0)
    

In [8]:
def dealing_with_no_work_experience_heading(file_path):
    line_number = 0
    List_Org = list()
    document = open(file_path, 'rb')
    #Create resource manager
    rsrcmgr = PDFResourceManager()
    fake_file_handle = io.StringIO()
    # Set parameters for analysis.
    laparams = LAParams(line_overlap=0.5, char_margin=50.0, line_margin=0.5,word_margin=0.1, boxes_flow=0.5,
                         detect_vertical=False, all_texts=False)
    # Create a PDF page aggregator object.
    device = PDFPageAggregator(rsrcmgr, laparams=laparams)
    interpreter = PDFPageInterpreter(rsrcmgr, device)
    
    for page in PDFPage.get_pages(document,caching=True,check_extractable=True):
        interpreter.process_page(page)
        # receive the LTPage object for the page.
        layout = device.get_result()
        for element in layout:
            line_number+=1
            if isinstance(element, LTTextBoxHorizontal):
                text=element.get_text()
                text=nlp_title(text)  
                lis=Spacy_Identified(text,nlp_title,line_number)
                if(lis != None):
                    List_Org.append(lis)  
    flatten=[l for subitem in List_Org for l in subitem]
    for l in flatten:
        year=calculate_duration(l[1])
        l[1].append(year)
    return(0)

In [9]:
#Function for converting pdf to text
def convert_pdf_to_text(file_path,nlp):
    
    start_time = time.time()
    counter, line_number_start, line_number_end = find_if_work_experience_available(file_path,nlp,start_time)
    
    if(counter == 1):
        score_1 = dealing_with_work_experience_heading(line_number_start,line_number_end,file_path,start_time)
    elif(counter == 0):
        score_1 = dealing_with_no_work_experience_heading(file_path)
        
    print("Score_1 for the resume: ", score_1)

In [10]:
convert_pdf_to_text("/data/home/devuser/Talenthub - Resume Analytics/July/Resumes_Test/Swathi's_Resume.pdf",nlp_title)

--- 4.729592561721802 seconds ---
--- 9.174578666687012 seconds ---
Score_1 for the resume:  0
